In [2]:
%pip install --upgrade llama-index        # core
%pip install --upgrade llama-index-llms-google-genai  # Google / Gemini LLM integration
%pip install --upgrade llama-index-embeddings-google-genai  # embeddings via Google GenAI
%pip install --upgrade google-generativeai  # underlying Google SDK

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
from pinecone import Pinecone
from llama_index.core import (
 Settings,
 VectorStoreIndex,
 SimpleDirectoryReader,
 KeywordTableIndex,)
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.query_engine import RouterQueryEngine,SubQuestionQueryEngine
from llama_index.core.question_gen import LLMQuestionGenerator
from llama_index.core.query_engine import SubQuestionQueryEngine

In [12]:
# ============================ 1) EMBEDDING / LLM LAYER ============================
# ============================ 2) DATA / INDEX LAYER ============================
# Keep LLM OFF while loading PDFs (prevents accidental calls)
# Set embedding BEFORE building Pinecone index
Settings.embed_model = GoogleGenAIEmbedding(
    model_name="models/text-embedding-004",
    api_key=""   # or your key directly
)

Settings.llm = None
try:
 docs = SimpleDirectoryReader("./GenAI/GenAI-NoteBooks/coffee_pages",required_exts=[".html"]).load_data()
except TypeError:
 docs = SimpleDirectoryReader("./GenAI/GenAI-NoteBooks/coffee_pages",file_exts=[".html"]).load_data()

kw_idx = KeywordTableIndex.from_documents(docs)

# Pinecone semantic index
pc = Pinecone(api_key="")
pc_index = pc.Index("coffeeindex")
vstore = PineconeVectorStore(pinecone_index=pc_index,text_key="text") # change if you used "page_content"
sem_idx = VectorStoreIndex.from_vector_store(vstore)

LLM is explicitly disabled. Using MockLLM.


In [13]:
# Turn LLM ON now
Settings.llm = GoogleGenAI(
 model="gemini-2.5-flash",
 api_key="",
 temperature=0.2,
 max_tokens=2024,
)

2025-11-29 15:00:35,421 - INFO - HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash "HTTP/1.1 200 OK"


In [14]:
# Wrap each index as a query engine (single-shot synthesis via response_mode="compact")
qe_sem = sem_idx.as_query_engine(similarity_top_k=5,response_mode="compact", llm=Settings.llm)
qe_kw = kw_idx.as_query_engine(response_mode="compact",llm=Settings.llm)

In [15]:
# Package as proper tools (use ToolMetadata, not dict)
tools = [
 QueryEngineTool(query_engine=qe_sem,metadata=ToolMetadata(name="semantic", description="General coffee KB(Pinecone)")),
 QueryEngineTool(query_engine=qe_kw,metadata=ToolMetadata(name="keyword", description="PDF keyword/headings lookup")),
]

In [16]:
# ============================ 3) QUERY LAYER============================
# (A) RouterQueryEngine -- route whole query to the best engine
router = RouterQueryEngine.from_defaults(
 query_engine_tools=tools,
 selector=LLMSingleSelector.from_defaults(llm=Settings.llm),
)
import nest_asyncio
nest_asyncio.apply()
print(router.query("Find entries mentioning ashwagandha in titles or headings.")) # likely routes to keyword
print(router.query("What is turmeric coffee?"))

2025-11-29 15:00:40,159 - INFO - AFC is enabled with max remote calls: 10.
2025-11-29 15:00:43,565 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-11-29 15:00:43,573 - INFO - Selecting query engine 1: The question asks to 'Find entries mentioning ashwagandha in titles or headings.' Choice (2) 'PDF keyword/headings lookup' directly addresses the need to look up keywords within headings, which is precisely what the question requires. Choice (1) 'General coffee KB(Pinecone)' is irrelevant as it pertains to coffee and a general knowledge base, not specific keyword/heading searches for 'ashwagandha'..
2025-11-29 15:00:43,575 - INFO - > Starting query: Find entries mentioning ashwagandha in titles or headings.
2025-11-29 15:00:43,577 - INFO - query keywords: ['find', 'answers', 'titles', 'avoid', 'provide', 'extracting', 'use', 'provided', 'keywords', 'lookup', 'given', 'question', 'extract', '10', 'm

The following entries mention ashwagandha in their titles or headings:

*   Ashwagandha Coffee (Adaptogenic Latte)
*   Mushroom & Ashwagandha Coffee (Adaptogen Blend)


2025-11-29 15:00:53,590 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-11-29 15:00:53,596 - INFO - Selecting query engine 0: The question 'What is turmeric coffee?' seeks a definition and description of a specific type of coffee. A 'General coffee KB (Pinecone)' implies a knowledge base containing information about various coffee types, ingredients, and preparations, making it the most direct source for answering such a question. 'PDF keyword/headings lookup' is a search method, not a knowledge source itself, and would require an existing PDF containing the information..
2025-11-29 15:00:55,703 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"


Empty Response


In [18]:
%pip install llama-index-question-gen-llm

ERROR: Could not find a version that satisfies the requirement llama-index-question-gen-llm (from versions: none)
ERROR: No matching distribution found for llama-index-question-gen-llm
Note: you may need to restart the kernel to use updated packages.


In [19]:
# (B) SubQuestionQueryEngine -- split, route parts, then merge
question_gen = LLMQuestionGenerator.from_defaults(llm=Settings.llm)
subq = SubQuestionQueryEngine.from_defaults(
 query_engine_tools=tools,
 question_gen=question_gen,
 llm=Settings.llm,
)
print(subq.query("Compare turmeric coffee vs saffron coffee and give a one-paragraph verdict."))

2025-11-29 15:02:39,661 - INFO - AFC is enabled with max remote calls: 10.
2025-11-29 15:02:42,075 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"


Generated 2 sub questions.
[semantic] Q: What are the characteristics and benefits of turmeric coffee?
[semantic] Q: What are the characteristics and benefits of saffron coffee?
[semantic] A: Empty Response


2025-11-29 15:02:44,371 - INFO - AFC is enabled with max remote calls: 10.


[semantic] A: Empty Response


2025-11-29 15:02:46,510 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"


A comparison between turmeric coffee and saffron coffee, including their characteristics and benefits, cannot be provided as the available information does not describe either beverage.
